In [1]:
import praw
import pandas as pd
import pprint
import numpy

CLIENT_ID = "GpWem_ykBPLXyEGKsaXFOg"
CLIENT_SECRET = "Or_ltwfIJnAbmYDvNkII6PXSJpFD1w"
USER_AGENT = "Middle_Practice8658"

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret = CLIENT_SECRET,
    user_agent=USER_AGENT
)

subreddit = reddit.subreddit('fentanyl')


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [3]:

# Fetch posts
posts = []
for submission in subreddit.new(limit = None):  # Adjust the limit as needed
    posts.append([submission.id, submission.title, submission.selftext, submission.score, submission.num_comments, submission.created_utc, submission.author, submission.url])

# Convert to DataFrame
posts_df = pd.DataFrame(posts, columns=['post_id', 'title', 'body', 'score', 'num_comments', 'created_utc', 'author', 'url'])

# Fetch comments
comments = []
for submission in subreddit.new(limit=998):  # Adjust the limit as needed
    submission.comments.replace_more(limit = 0) #this limit is how many times the API will click "more comments" -- sadly, we can't do all, as we get a "too many requests" error
    for comment in submission.comments.list():
        comments.append([comment.id, comment.body, comment.score, comment.created_utc, comment.author, submission.id])

# Convert to DataFrame
comments_df = pd.DataFrame(comments, columns=['id', 'body', 'score', 'created_utc', 'author', 'post_id'])

import ftfy

# this will fix your encoding problem
posts_df['body'] = posts_df['body'].map(ftfy.fix_encoding)
comments_df['body'] = comments_df['body'].map(ftfy.fix_encoding)

merged_df = comments_df.merge(posts_df, left_on='post_id', right_on='post_id', suffixes=('_comment', '_post'))


In [5]:
#add to file 
# 
import os
# add to existing CSV file
csv_file = 'PRAW_r_fentanyl.csv'

if os.path.exists(csv_file):
    existing_posts_df = pd.read_csv(csv_file)
    combined_df = pd.concat([existing_posts_df, merged_df]).drop_duplicates(subset='id', keep='last')
    combined_df.to_csv(csv_file, index=False)
else:
    merged_df.to_csv(csv_file, index=False, encoding='utf-8')